## Phase 2: Feature Engineering & Customer Segmentation

We define an observation window ending on a fixed cutoff date.
All features are computed using data **before** this cutoff.
Churn will be evaluated **after** this cutoff to avoid data leakage.


In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv(
    "../data/raw/online_retail.csv",
    encoding="ISO-8859-1"
)

df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])
df = df.dropna(subset=["Customer ID"])

CUT_OFF_DATE = df["InvoiceDate"].max() - pd.Timedelta(days=30)
CUT_OFF_DATE


Timestamp('2011-11-09 12:50:00')

In [3]:
obs_df = df[df["InvoiceDate"] <= CUT_OFF_DATE].copy()


All customer features are derived strictly from data prior to the cutoff date.


In [6]:
obs_df["TotalPrice"] = obs_df["Quantity"] * obs_df["Price"]
rfm = (
    obs_df.groupby("Customer ID")
    .agg(
        recency=("InvoiceDate", lambda x: (CUT_OFF_DATE - x.max()).days),
        frequency=("Invoice", "nunique"),
        monetary=("TotalPrice", "sum")
    )
)

rfm.head()


,recency,frequency,monetary
Customer ID,,,
12346.0,295,17,-64.68
12347.0,9,7,5408.50
12348.0,44,5,2019.40
12349.0,377,4,2646.99
12350.0,279,1,334.40


In [7]:
rfm["R_score"] = pd.qcut(rfm["recency"], 5, labels=[5,4,3,2,1])
rfm["F_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1,2,3,4,5])
rfm["M_score"] = pd.qcut(rfm["monetary"], 5, labels=[1,2,3,4,5])

rfm["RFM_score"] = (
    rfm["R_score"].astype(int) +
    rfm["F_score"].astype(int) +
    rfm["M_score"].astype(int)
)


In [8]:
def segment_customer(row):
    if row["RFM_score"] >= 13:
        return "High Value - Loyal"
    elif row["R_score"] <= 2 and row["F_score"] >= 4:
        return "High Value - At Risk"
    elif row["R_score"] >= 4 and row["F_score"] <= 2:
        return "New / Low Engagement"
    elif row["RFM_score"] <= 6:
        return "Low Value - Likely Churn"
    else:
        return "Mid Value - Stable"


In [9]:
rfm["segment"] = rfm.apply(segment_customer, axis=1)
rfm["segment"].value_counts()


segment
Mid Value - Stable          2021
Low Value - Likely Churn    1642
High Value - Loyal          1252
New / Low Engagement         503
High Value - At Risk         362
Name: count, dtype: int64

In [10]:
segment_summary = (
    rfm.groupby("segment")
    .agg(
        customers=("RFM_score", "count"),
        avg_revenue=("monetary", "mean"),
        avg_frequency=("frequency", "mean"),
        avg_recency=("recency", "mean")
    )
    .sort_values("avg_revenue", ascending=False)
)

segment_summary


,customers,avg_revenue,avg_frequency,avg_recency
segment,,,,
High Value - Loyal,1252,9150.686270,21.138179,28.261981
High Value - At Risk,362,2524.043594,8.765193,354.102210
Mid Value - Stable,2021,1242.111571,4.514102,149.738743
New / Low Engagement,503,491.389744,1.427435,26.479125
Low Value - Likely Churn,1642,221.936956,1.352010,424.059074


In [11]:
rfm.to_csv("../data/processed/rfm_features.csv")
